In [1]:
import requests
import time
import pandas as pd


class FipeClient:
    """
    Cliente para consumir endpoints da FIPE:
      - marcas
      - modelos
      - anos/modelo
      - valor com todos parâmetros
    Implementa retry exponencial para lidar com falhas de rede ou 429.
    """

    BASE_URLS = {
        'marcas': 'https://veiculos.fipe.org.br/api/veiculos/ConsultarMarcas',
        'modelos': 'https://veiculos.fipe.org.br/api/veiculos/ConsultarModelos',
        'anos': 'https://veiculos.fipe.org.br/api/veiculos/ConsultarAnoModelo',
        'valor': 'https://veiculos.fipe.org.br/api/veiculos/ConsultarValorComTodosParametros'
    }

    def __init__(self, tabela_referencia: int = 321, tipo_veiculo: int = 1):
        self.referencia = tabela_referencia
        self.tipo_veiculo = tipo_veiculo
        self.session = requests.Session()

    def _post(self, url: str, payload: dict, max_retries: int = 4) -> dict:
        """
        Faz POST com retry exponencial (1s, 2s, 4s).
        Lança exceção se falhar após max_retries.
        """
        delay = 1
        for attempt in range(1, max_retries + 1):
            try:
                resp = self.session.post(url, data=payload, timeout=10)
                if resp.status_code == 200:
                    return resp.json()
                elif resp.status_code == 429:
                    time.sleep(delay)
                else:
                    resp.raise_for_status()
            except requests.RequestException:
                # última tentativa: propaga
                if attempt == max_retries:
                    raise
                time.sleep(delay)
            delay *= 2  # exponencial
        # nunca chega aqui
        return {}

    def get_marcas(self) -> list[int]:
        """Retorna lista de códigos de marca."""
        payload = {
            'codigoTabelaReferencia': self.referencia,
            'codigoTipoVeiculo': self.tipo_veiculo
        }
        data = self._post(self.BASE_URLS['marcas'], payload)
        return [item['Value'] for item in data]

    def get_modelos(self, codigo_marca: int) -> dict[int, list]:
        """Retorna dict {codigo_modelo: []} para popular mais tarde com anos/combustível."""
        payload = {
            'codigoTabelaReferencia': self.referencia,
            'codigoTipoVeiculo': self.tipo_veiculo,
            'codigoMarca': codigo_marca
        }
        data = self._post(self.BASE_URLS['modelos'], payload)
        modelos = data.get('Modelos', [])
        return {item['Value']: [] for item in modelos}

    def get_anos_combustivel(self, codigo_marca: int, codigo_modelo: int) -> list[tuple[int, int]]:
        """Retorna lista de tuples (ano, tipo_combustível)."""
        payload = {
            'codigoTabelaReferencia': self.referencia,
            'codigoTipoVeiculo': self.tipo_veiculo,
            'codigoMarca': codigo_marca,
            'codigoModelo': codigo_modelo
        }
        data = self._post(self.BASE_URLS['anos'], payload)
        return [
            tuple(map(int, entry['Value'].split('-')))
            for entry in data
        ]

    def get_valores(self, response_modelo: dict[int, dict[int, list[tuple[int, int]]]]) -> pd.DataFrame:
        """
        Itera sobre todos os (marca, modelo, ano, combustivel) e
        retorna um DataFrame com os JSONs de resposta.
        """
        records = []
        for marca, modelos in response_modelo.items():
            for modelo, anos in modelos.items():
                for ano, comb in anos:
                    payload = {
                        'codigoTabelaReferencia': self.referencia,
                        'codigoTipoVeiculo': self.tipo_veiculo,
                        'codigoMarca': marca,
                        'codigoModelo': modelo,
                        'anoModelo': ano,
                        'codigoTipoCombustivel': comb,
                        'tipoVeiculo': 'carro',
                        'modeloCodigoExterno': None,
                        'tipoConsulta': 'tradicional'
                    }
                    json_data = self._post(self.BASE_URLS['valor'], payload)
                    records.append(json_data)
        return pd.DataFrame(records)



In [ ]:
client = FipeClient()
    # 1) marcas
marcas = client.get_marcas()
print(marcas)

['1', '2', '3', '4', '5', '189', '6', '207', '7', '8', '123', '238', '236', '10', '245', '161', '11', '136', '182', '12', '13', '14', '241', '15', '16', '246', '17', '147', '18', '19', '20', '249', '21', '149', '22', '190', '170', '199', '23', '153', '24', '240', '152', '214', '25', '26', '27', '208', '177', '28', '29', '154', '30', '31', '32', '171', '33', '34', '168', '127', '35', '140', '36', '37', '38', '211', '39', '40', '167', '156', '41', '42', '250', '43', '44', '45', '46', '47', '185', '186', '48', '195', '49', '50', '51', '52', '247', '183', '157', '125', '54', '55', '165', '56', '57', '58', '59', '163', '120']


: 